# Labelling street furniture

E.g. runnish bins and city benches.

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import time
import logging

import upcp.utils.las_utils as las_utils
import upcp.utils.log_utils as log_utils
import upcp.utils.csv_utils as csv_utils
import upcp.utils.ahn_utils as ahn_utils
import upcp.utils.bgt_utils as bgt_utils
import upcp.fusion as fusion
from upcp.labels import Labels
from upcp.pipeline import Pipeline

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

## Street furniture fuser based on cluster dimensions and BGT reference data
First, download and parse the required BGT data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb).

In [ ]:
# The already labelled points must be removed.
exclude_labels = (Labels.GROUND, Labels.ROAD, Labels.BUILDING, Labels.TREE, Labels.STREET_LIGHT,
                  Labels.TRAFFIC_SIGN, Labels.CAR, Labels.NOISE)

In [ ]:
# Select the file to process.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# Data folder for the fusers.
bgt_data_file = '../datasets/bgt/bgt_street_furniture_demo.csv'
bgt_street_furniture_reader = bgt_utils.BGTPointReader(bgt_file=bgt_data_file)

# We need elevation data to determine object height above ground.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
# Object type-specific parameters.
rubbish_bin_params = {'min_height': 0.5, 'max_height': 1.5, 'min_width': 0.3, 'max_width': 1.5, 'min_length': 0.3, 'max_length': 1.5}
city_bench_params = {'min_height': 0.5, 'max_height': 1.5, 'min_width': 0.3, 'max_width': 2.0, 'min_length': 1.0, 'max_length': 3.5}

# Create fusers for BGT point data.
rubbish_bin_fuser = fusion.BGTStreetFurnitureFuser(Labels.RUBBISH_BIN, bgt_type='afvalbak', bgt_reader=bgt_street_furniture_reader,
                                                   ahn_reader=ahn_reader, params=rubbish_bin_params)
city_bench_fuser = fusion.BGTStreetFurnitureFuser(Labels.CITY_BENCH, bgt_type='bank', bgt_reader=bgt_street_furniture_reader,
                                                  ahn_reader=ahn_reader, params=city_bench_params)

## Process point clouds and save the results
The pipeline can now be used to process point clouds. Labels will be written to the LAS file as a new extended field called label.

In [ ]:
# Set-up the order in how we want to label.
process_sequence = (rubbish_bin_fuser, city_bench_fuser)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

## Process a single file

In [ ]:
# Process the file.
pipeline.process_file(in_file, out_file=out_file)